In [1]:
import numpy as np
import pandas as pd
import requests 
import xlsxwriter
import math

In [57]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [9]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 86393524, 'calculationPrice': 'close', 'change': -3.21, 'changePercent': -0.02087, 'close': 154.22, 'closeSource': 'folcaiif', 'closeTime': 1719824387977, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 160.56, 'delayedPriceTime': 1733026073948, 'extendedChange': -1.29, 'extendedChangePercent': -0.00822, 'extendedPrice': 153.04, 'extendedPriceTime': 1721912267816, 'high': 166.38, 'highSource': 'ird  unaieepcleye 5mtd1', 'highTime': 1701654739299, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 157.16, 'iexCloseTime': 1664200168084, 'iexLastUpdated': 1669612206369, 'iexMarketPercent': 0.019899649063215185, 'iexOpen': 158.48, 'iexOpenTime': 1684647946374, 'iexRealtimePrice': 154.67, 'iexRealtimeSize': 101, 'iexVolume': 2051340, 'lastTradeTime': 1708634048669, 'latestPrice': 157.23, 'latestSource': 'Close', 'latestTime': 'September 21, 2022', 'latestUpdate': 1728559005067, 'latestVolume': 106304352, 'low': 154.22, 'lowSou

In [12]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap)

2558542516672


In [17]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [26]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,157.23,2558542516672,N/A


In [33]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index = my_columns
        ),
        ignore_index = True
    )

In [34]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.20,38345182693,N/A
1,AAL,13.60,8945542736,N/A
2,AAP,167.30,10181027917,N/A
3,AAPL,157.42,2556250859152,N/A
4,ABBV,145.21,250795909900,N/A


In [35]:
def chunks(lst, n):
    for i in range (0, len(lst), n):
        yield lst[i : i + n]

In [58]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )
        
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.35,38578095368,N/A
1,AAL,13.83,8956939207,N/A
2,AAP,168.30,10018016527,N/A
3,AAPL,155.07,2516298951352,N/A
4,ABBV,141.88,253953118285,N/A
...,...,...,...,...
496,YUM,114.11,32656400199,N/A
497,ZBH,114.20,24207156015,N/A
498,ZBRA,285.26,15187431481,N/A
499,ZION,58.40,8601101383,N/A


In [90]:
portfolio_size = input('Enter portfolio value: ')

position_size = val/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])

final_dataframe

Enter portfolio value: 1000000


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.35,38578095368,156
1,AAL,13.83,8956939207,1443
2,AAP,168.30,10018016527,118
3,AAPL,155.07,2516298951352,128
4,ABBV,141.88,253953118285,140
...,...,...,...,...
496,YUM,114.11,32656400199,174
497,ZBH,114.20,24207156015,174
498,ZBRA,285.26,15187431481,69
499,ZION,58.40,8601101383,341


In [91]:
writer = pd.ExcelWriter('Trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Trades', index = False)

In [92]:
string_format = writer.book.add_format(
        {
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'border': 1
        }
    )

In [93]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [94]:

writer.save()